In [8]:
! pip install sentence_transformers
import PyPDF2
import pandas as pd
import os
import re
import numpy as np
from os import listdir
from os.path import isfile, join
from io import StringIO
import pandas as pd
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()
from spacy.matcher import PhraseMatcher
from sentence_transformers import SentenceTransformer, util
import numpy as np
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')
import re
from re import sub
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

You should consider upgrading via the 'c:\users\yskvi\anaconda3\python.exe -m pip install --upgrade pip' command.


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\yskvi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [13]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\yskvi\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [14]:
from PyPDF2 import PdfFileReader

In [21]:
class RelevanceScore:
    
    def __init__(self,resume_name,jd_name):
        self.resume_name = resume_name
        self.jd_name = jd_name
    
    def pdfTextExtract(self,path):
        with open(path, 'rb') as f:
            pdf = PdfFileReader(f)
            countpage=pdf.getNumPages()
            count=0
            jd_text=[]

            while count < countpage:    
                pageObj = pdf.getPage(count)
                count +=1
                t = pageObj.extractText()

                jd_text.append(t)
            return jd_text


    def conversion(self,file):
        text = self.pdfTextExtract(file) 
        text = str(text)
        text = text.replace("\\n", "")
        text = text.lower()
        cleaned_text=re.split(' ; | , | . | | : | / | - ',text)
        bad_chars = [';', ':', '!', "*",",",".","[",'"',"'","]"]

        for i in range(len(cleaned_text)):
            for j in bad_chars:
                cleaned_text[i]=cleaned_text[i].replace(j,'')
        stop_words = list(set(stopwords.words('english')))

#         for i in cleaned_text:
#             if i in stop_words:
#                 cleaned_text.remove(i)
        cleaned_text=[i for i in cleaned_text if i]
        cleaned_text=' '.join([str(elem) for elem in cleaned_text])
        print("cleaned_text ",cleaned_text)
        return cleaned_text

    def PrintStats(self,resume_name,text1):
        keyword_dict = pd.read_csv('template.csv')
        
        front_end_words = [nlp(text) for text in keyword_dict['front_end'].dropna(axis = 0)]
        back_end_words = [nlp(text) for text in keyword_dict['back_end'].dropna(axis = 0)]
        de_words = [nlp(text) for text in keyword_dict['data_engineer'].dropna(axis = 0)]
        ds_words = [nlp(text) for text in keyword_dict['data_science'].dropna(axis = 0)]

        matcher = PhraseMatcher(nlp.vocab,attr='LOWER')
        matcher.add('FrontEnd', None, *front_end_words)
        matcher.add('BackEnd', None, *back_end_words)
        matcher.add('DE', None, *de_words)
        matcher.add('DS', None, *ds_words)
        doc = nlp(text1)
        
        d = []  
        matches = matcher(doc)
        for match_id, start, end in matches:
            rule_id = nlp.vocab.strings[match_id]
            span = doc[start : end]
            d.append((rule_id, span.text))      
        keywords = "\n".join(f'{i[0]} {i[1]} ({j})' for i,j in Counter(d).items())

        df = pd.read_csv(StringIO(keywords),names = ['Keywords_List'])
        df1 = pd.DataFrame(df.Keywords_List.str.split(' ',1).tolist(),columns = ['Subject','Keyword'])
        df2 = pd.DataFrame(df1.Keyword.str.split('(',1).tolist(),columns = ['Keyword', 'Count'])
        df3 = pd.concat([df1['Subject'],df2['Keyword'], df2['Count']], axis =1) 
        df3['Count'] = df3['Count'].apply(lambda x: x.rstrip(")"))

  
        filename = self.resume_name
        name = filename.split('_')
        name2 = name[0]
        name2 = name2.lower()
        name3 = pd.read_csv(StringIO(name2),names = ['Candidate Name'])

        dataf = pd.concat([name3['Candidate Name'], df3['Subject'], df3['Keyword'], df3['Count']], axis = 1)
        dataf['Candidate Name'].fillna(dataf['Candidate Name'].iloc[0], inplace = True)
    
        print(dataf)


    def relevance(self,resume,jd):
        model = SentenceTransformer('stsb-roberta-large')
        def sent_simi_score(sent1, sent2):
            #encode sentences to get their embeddings
            embedding1 = model.encode(sent1, convert_to_tensor=True)
            embedding2 = model.encode(sent2, convert_to_tensor=True)

            # compute similarity scores of two embeddings
            cosine_scores = util.pytorch_cos_sim(embedding1, embedding2)

            return cosine_scores.item()
        
        return sent_simi_score(resume,jd)


    def start_method(self):
        
        text1 = self.conversion(resumes_path+"\\"+self.resume_name)
        text2 = self.conversion(jd_path+"\\"+self.jd_name)
        #print("tefgreg",text1,"fsrf",text2)
        relevance_score = self.relevance(text1,text2)
        stats=self.PrintStats(self.resume_name,text1)
        return relevance_score,text1,text2,stats



In [24]:
resumes_path=os.getcwd()+"\\resumes"
jd_path=os.getcwd()+"\\jd"
x=RelevanceScore('resume2.pdf','jobdescription2.pdf')
a,b,c,d=x.start_method()
print('relevance score= ',a)
print()
print('resume text ',b)
print()
print("jd text ",c)
print()

cleaned_text  sai krishna vivek +91-9701508517 linkedincom/in/yskvivek yskvivek9@gmailcom profile 2+ years experience data engineering machine learning using spark hadoop technologies strong knowledge financial analytics banking domain currently pursuing masters analytics georgia tech special electives finance time series modelling excellent organizational analytical problem-solving presentation skills experience handling multiple stake holders proactive fast learner committed learning new technologies well adept switching domains fast learning curve responsible developing first its kind analytics product finance is completely based open source technologies responsible developing efficient multilevel chemical structure-based search web application million-compound dataset csir iict part research internship education master science analytics (online) 2021(expected) georgia institute technology (35 gpa) bachelor technology (computer science engineering) 2018 gokaraju rangaraju institute 